In [1]:
!pip install folium
import folium
import requests
!pip install geopy
import geopy
import numpy as np
import pandas as pd

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


#### Using the read_html function in pandas to create the dataframe

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='html5lib')

In [3]:
df = df[0]
pc = df[0]
bors = df[1]
neigh = df[2]
pc.drop(0,inplace=True)
bors.drop(0,inplace=True)
neigh.drop(0,inplace=True)

#### Creating a new dataframe to store all columns

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df_codes = pd.DataFrame(columns=cols)

In [5]:
df_codes['PostalCode'] = pc
df_codes['Borough'] = bors
df_codes['Neighborhood'] = neigh

#### Removing the elements whose boroughs have not been assigned

In [6]:
df_codes.drop(df_codes[df_codes.Borough == "Not assigned"].index, inplace=True)

df_codes = df_codes.reset_index()

df_codes.drop(['index'], axis=1, inplace=True)

df_codes.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Copying the borough name to the neighborhood when it is not assigned

In [7]:
for i in range(len(df_codes)):
    if (df_codes['Neighborhood'][i] == "Not assigned"):
        df_codes['Neighborhood'][i] = df_codes['Borough'][i]
df_codes.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Loop for grouping by postalcodes and merging same postalcode cells
##### Inds stores the indexes of the rows merged(which will be dropped later)

In [8]:
inds = []
for i in range(0,(len(df_codes)-1)):
    if( str(df_codes['PostalCode'].iloc[i]) == str(df_codes['PostalCode'].iloc[i+1])):
        df_codes['Neighborhood'].iloc[i] += ', '
        df_codes['Neighborhood'].iloc[i] += df_codes['Neighborhood'].iloc[i+1]
        inds.append(int(i+1))
df_codes.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
9,M1B,Scarborough,Malvern


In [9]:
df_codes.drop(inds, inplace=True)

In [10]:
df_codes = df_codes.reset_index()
df_codes.drop(['index'], axis=1, inplace=True)
df_codes.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df_codes['Latitude'] = pd.Series(np.random.randn(len(df_codes)), index=df_codes.index)
df_codes['Longitude'] = pd.Series(np.random.randn(len(df_codes)), index=df_codes.index)

### Using the csv file (for latitudes and longitudes) because geocoder could not get all values 

In [12]:
df2 = pd.read_csv('D:\DataSets\Geospatial_Coordinates.csv')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The following cell gives the index of the row satisfying a given condition

In [13]:
df2[df2['Postal Code'] == 'M4A'].index.values.astype(int)[0]

34

As df2 and df_codes are not arranged in the same order, therefore we have to find the index of the row in df2 having the same PostalCode as that in df_codes

In [14]:
for i in range(len(df_codes)):
    ind = df2[df2['Postal Code'] == str(df_codes['PostalCode'].iloc[i])].index.values.astype(int)[0]
    df_codes['Latitude'].iloc[i] = float(df2['Latitude'].iloc[ind])
    df_codes['Longitude'].iloc[i] = float(df2['Longitude'].iloc[ind])

C:\Python27\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
df_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Clustering

#### Latitude and Longitude of Toronto

In [16]:
TorLat = 43.6532
TorLon = -79.3832

In [17]:
mapTor = folium.Map(location=[TorLat, TorLon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_codes['Latitude'], df_codes['Longitude'], df_codes['Borough'], df_codes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(mapTor)  

    
mapTor

In [18]:
import requests
import json

In [19]:
CLIENT_ID = 'X2VYAOTLOCJK4X5CUB45HEUI2KD2HIBYKCMFU5FZTXSLLSDT' 
CLIENT_SECRET = 'QVFVSX2UNS0UN0MSDUM5TCS310EHHBI15Z2REYD2OLMSCHAR' 
VERSION = '20181221'

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
Torvenues = getNearbyVenues(names = df_codes['Neighborhood'],
                            latitudes = df_codes['Latitude'],
                            longitudes = df_codes['Longitude'])

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington
Highland Creek, Rouge Hill
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate
Guildwood, Morningside
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North
Thorncliffe Park
Adelaide, King
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands
Little Portugal, Trinity
East Birchmount Park, Ionview
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place
Clairlea, Golden Mile
Silver Hills, York Mills
Downsview West
The Beaches Wes

In [22]:
print(Torvenues.shape)
Torvenues.head()

(2248, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [23]:
Torvenues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
Berczy Park,54,54,54,54,54,54


In [24]:
Toronehot = pd.get_dummies(Torvenues[["Venue Category"]], prefix="", prefix_sep = "")
Toronehot['Neighborhood'] = Torvenues['Neighborhood']
Toronehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
Torgrped = Toronehot.groupby('Neighborhood').mean().reset_index()

In [26]:
Torgrped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.010000,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.010000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.058824,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


In [27]:
from sklearn.cluster import KMeans

In [28]:
clusters = 5
Torclustering = Torgrped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(Torclustering)
kmeans.labels_[0:10]

array([2, 2, 1, 0, 0, 2, 2, 2, 2, 2])

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Torgrped['Neighborhood']

for ind in np.arange(Torgrped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Torgrped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Gym,Bakery,Clothing Store,Hotel
1,Agincourt,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East",Playground,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
3,"Albion Gardens, Beaumond Heights",Pizza Place,Grocery Store,Fried Chicken Joint,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Golf Course,Dance Studio
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pool,Pub,Sandwich Place,Skating Rink,Pharmacy,Gym,Gift Shop
5,"Bathurst Manor, Downsview North",Coffee Shop,Pizza Place,Frozen Yogurt Shop,Shopping Mall,Fast Food Restaurant,Bridal Shop,Sandwich Place,Diner,Bank,Sushi Restaurant
6,Bayview Village,Bank,Café,Chinese Restaurant,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Sushi Restaurant,Grocery Store,Butcher,Restaurant,Café,Pub
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Steakhouse,Bakery,Italian Restaurant,Seafood Restaurant,Farmers Market,Pub,Cheese Shop
9,"Birch Cliff, Cliffside West",College Stadium,Café,General Entertainment,Skating Rink,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [31]:
Tormerged = neighborhoods_venues_sorted
Tormerged["Cluster Labels"] = kmeans.labels_

Tormerged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,"Adelaide, King",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Gym,Bakery,Clothing Store,Hotel,2
1,Agincourt,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,2
2,"Agincourt North, L'Amoreaux East",Playground,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,1
3,"Albion Gardens, Beaumond Heights",Pizza Place,Grocery Store,Fried Chicken Joint,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Golf Course,Dance Studio,0
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pool,Pub,Sandwich Place,Skating Rink,Pharmacy,Gym,Gift Shop,0


In [32]:
kmeans.labels_.shape

(99,)

#### Neighborhoods with cluster labels

In [33]:
Tormerged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,"Adelaide, King",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Gym,Bakery,Clothing Store,Hotel,2
1,Agincourt,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,2
2,"Agincourt North, L'Amoreaux East",Playground,Park,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,1
3,"Albion Gardens, Beaumond Heights",Pizza Place,Grocery Store,Fried Chicken Joint,Pharmacy,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Golf Course,Dance Studio,0
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Athletics & Sports,Pool,Pub,Sandwich Place,Skating Rink,Pharmacy,Gym,Gift Shop,0
